# Long Term Stock Performance Prediction

## Business Understanding and Overview

For decades, many humans have attempted to predict the stock market. These attempts have varied in nature from predicting individual stock prices to predicting the direction of the overall market. While most argue that it's impossible to beat the market as an individual investor, I believe that with the right tools and technology, individuals can indeed outperform the market. While I believe this idea is possible, the data shows that individual investors' performances are dreadful on average when compared to the broader market. The bar chart below shows that the average investor had annual returns of 1.9% from 1999-2018 while the S&P500 returned 5.6%.

 ![](images/IndInvInfographic.jpeg)

Currently, to be a well-versed individual investor requires significant time researching companies, listening to earnings calls, and having an understanding of macro-economic trends. Most people do not have the time to become knowledgable on multiple companies that are publicly traded, so it's easier and safer to invest in index funds that track the overall market. But what if there was a tool that could quickly let individual investors know which companies can beat the market over an extended period of time?


## Data Understanding

I have sourced all the required data myself using the SEC's API. I used this API to pull the annual reports (10-K) for public companies from 2009-2012. I could not get any useful data from before 2009 because the SEC didn't require companies to file in XBRL format until then. In the future, I plan to use other methods to acquire and clean data prior to 2009. I stopped collecting data past 2012 because I need a 10 year window to determine if a stock outperformed the market over 10 years based on their financials. 

In terms of metrics that I will use to score the performance of my model, recall will be the most important metric followed by precision. Recall will score how my model does at identifying stocks that do outperform the market. Investors will care more about finding the stocks that can outperform the market rather than correctly identifying those that underperform. Precision will also be a key metric because this gives a score for how accurate our model performs when predicting that a stock does outperform the market. To simplify the metrics I use, I can also use the f1 score as this is the harmonic mean of precision and recall.


In [1]:
# import all relevant libraries
from IPython.display import Image

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import pickle
import time
import requests
import json
from sec_api import QueryApi

import yfinance as yf
from sec_cik_mapper import StockMapper

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)

I am using base code from https://gist.github.com/janlukasschroeder/3da274150fd00c1c1776c7e541a9b61b to get me started in scraping financial statements from the SEC. I will be using 10-K forms (annual SEC filing) rather than all quarterly filings (10-Q) to reduce the size of data.

This Github user's project gave me base code for functions that convert xbrl data to json format, pull relevant financial documents (balance sheet, income statement, etc.), and run queries through the SEC API.

In [3]:
# XBRL-to-JSON converter API endpoint
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

In [4]:
# convert XBRL-JSON of balance sheet to pandas dataframe
def get_balance_sheet(xbrl_json):
    
    """
    This function takes in the json data for a given SEC filing and returns a pandas dataframe of \
    the financial information relevant to the balance sheet.
    """
    
    # create an empty dictionary to begin storing financial info
    balance_sheet_store = {}

    # loop through each financial item in the BalanceSheets portion of the json data
    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in xbrl_json['BalanceSheets'][usGaapItem]:
            # only consider items without segment.
            if 'segment' not in fact:
                index = fact['period']['instant']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue
                    
                # add 0 if value is nil
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    
            
            # create a pandas series for each financial item
            balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies) 

    # combine all pandas series into a dataframe        
    balance_sheet = pd.DataFrame(balance_sheet_store).reset_index()
    # create a column that is the year for the period the financials are reported for
    balance_sheet['index'] = balance_sheet['index'].str.slice(-10, -6)
    balance_sheet = balance_sheet.rename(columns={'index': 'year'})
    balance_sheet = balance_sheet.sort_values(by='year')
    # create a column for the central index key, as we will use this to get the stock ticker symbol later
    balance_sheet['CIK'] = xbrl_json['CoverPage']['EntityCentralIndexKey']
    balance_sheet = pd.DataFrame(balance_sheet.iloc[-1, :]).T

    return balance_sheet

In [5]:
#### convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    
    """
    This function takes in the json data for a given SEC filing and returns a pandas dataframe of \
    the financial information relevant to the income statement.
    """
    
    # create an empty dictionary to begin storing financial info
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    
        
        # save the items for each financial data piece into a pandas series
        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    # combine all pandas series into a dataframe
    income_statement = pd.DataFrame(income_statement_store).reset_index()
    # pull out the year from the date range and create a new column for the year
    income_statement['index'] = income_statement['index'].str.slice(-10, -6)
    income_statement = income_statement.rename(columns={'index': 'year'})
    income_statement = income_statement.sort_values(by='year')
    # create a column for the Central Index Key as this will be used to get stock ticker symbols later on 
    income_statement['CIK'] = xbrl_json['CoverPage']['EntityCentralIndexKey']
    income_statement = pd.DataFrame(income_statement.iloc[-1, :]).T
    
    return income_statement



In [6]:
def get_cash_flow_statement(xbrl_json):
    
    """
    This function takes in the json data for a given SEC filing and returns a pandas dataframe of \
    the financial information relevant to the cash flow statement.
    """
    
    # create empty dictionary to begin storing relevant financial info
    cash_flows_store = {}

    # iterate through each financial item in the cash flow statement
    for usGaapItem in xbrl_json['StatementsOfCashFlows']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfCashFlows'][usGaapItem]:        
            # only consider items without segment.
            if 'segment' not in fact:
                # check if date instant or date range is present
                if "instant" in fact['period']:
                    index = fact['period']['instant']
                else:
                    index = fact['period']['startDate'] + '-' + fact['period']['endDate']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue

                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    
        
        # create a pandas series for each financial item
        cash_flows_store[usGaapItem] = pd.Series(values, index=indicies) 

    # combine all pandas series into one dataframe
    cash_flows = pd.DataFrame(cash_flows_store).reset_index()
    # create a column for the year from which the financial data was reported
    cash_flows['index'] = cash_flows['index'].str.slice(-10, -6)
    cash_flows = cash_flows.rename(columns={'index': 'year'})
    cash_flows = cash_flows.sort_values(by='year')
    # create a column for the Central Index Key as this will be used to get stock ticker symbols later on
    cash_flows['CIK'] = xbrl_json['CoverPage']['EntityCentralIndexKey']
    cash_flows = pd.DataFrame(cash_flows.iloc[-2, :]).T
    
    return cash_flows

In [7]:
import datetime

# create a start and end date to collect financial documents
start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")
end = datetime.datetime.strptime("2021-12-31", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    
# only take every 7 days (weekly intervals)
cleaned_dates = []
for date in date_generated[::7]:
    cleaned_dates.append(date.strftime("%Y-%m-%d"))
    
cleaned_dates[:5]

['2021-01-01', '2021-01-08', '2021-01-15', '2021-01-22', '2021-01-29']

In [8]:
# create a list of date ranges that we can run our query on 
cleaned_date_range = [f"{cleaned_dates[i]} TO {cleaned_dates[i+1]}" for i in range(len(cleaned_dates) - 1)]
cleaned_date_range[:5]

['2021-01-01 TO 2021-01-08',
 '2021-01-08 TO 2021-01-15',
 '2021-01-15 TO 2021-01-22',
 '2021-01-22 TO 2021-01-29',
 '2021-01-29 TO 2021-02-05']

In [9]:
my_tickers = pd.read_excel('BUDGET.xlsx', sheet_name='Stock Allocation')

In [16]:
ticker_list = list(my_tickers['Ticker'].dropna())
ticker_list

['MELI',
 'AMZN',
 'CRWD',
 'NVDA',
 'SHOP',
 'LULU',
 'ROKU',
 'SWAV',
 'SQ',
 'TWLO',
 'PYPL',
 'LMND',
 'MDB',
 'ACN',
 'TEAM',
 'ASML',
 'KNSL',
 'PUBM',
 'U',
 'DIS',
 'MGNI',
 'PINS',
 'MA',
 'TDOC',
 'AMD',
 'FLGT',
 'AMT',
 'LTCH',
 'ESTC',
 'V',
 'DAVA',
 'ILMN',
 'FVRR',
 'PI',
 'TWST',
 'COIN',
 'MASI',
 'MMM',
 'DOCU',
 'AYX',
 'CFLT',
 'FSLY',
 'TER',
 'OPAD',
 'WIX',
 'STNE',
 'STOR',
 'CHWY',
 'ARBK',
 'SEMR',
 'JMIA',
 'SKLZ',
 'UBX']

In [80]:
full_query = []
query_api = QueryApi(api_key='88140c3a554da7220ba7453a5e1ea2221c5f1e0e42c3293bbe75857245774e2e')

# loop through the date range created and pull all 10-K forms that were reported within the date range
for ticker in ticker_list:
    query = {
        "query": {
            "query_string": {
                "query": f"(formType:\"10-K\") AND (ticker:{ticker})"
                             }
                },
        "size": "200",
        "sort": [{ "filedAt": { "order": "desc" } }]
        }

    full_query.append(query_api.get_filings(query)['filings'])

In [81]:
#create an empty list to fill with accession numbers
accession_numbers = []

# extract accession numbers of each filing
for filing in full_query:
    for item in filing:
        accession_numbers.append(item['accessionNo']);

In [82]:
full_query

[[{'id': '436b5ee548ca16a833420bd59ff84299',
   'accessionNo': '0001562762-22-000049',
   'cik': '1099590',
   'ticker': 'MELI',
   'companyName': 'MERCADOLIBRE INC',
   'companyNameLong': 'MERCADOLIBRE INC (Filer)',
   'formType': '10-K',
   'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]',
   'filedAt': '2022-02-23T16:02:19-05:00',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1099590/000156276222000049/0001562762-22-000049.txt',
   'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1099590/000156276222000049/0001562762-22-000049-index.htm',
   'linkToXbrl': '',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1099590/000156276222000049/meli-20211231x10k.htm',
   'entities': [{'companyName': 'MERCADOLIBRE INC (Filer)',
     'cik': '1099590',
     'irsNo': '980212790',
     'fiscalYearEnd': '1231',
     'type': '10-K',
     'act': '34',
     'fileNo': '001-33647',
     'filmNo': '22663383',
     'sic': '7389 Services-

In [83]:
len(full_query)

53

In [84]:
len(accession_numbers)

489

In [85]:
import time

# get XBRL-JSON for a given accession number
def get_xbrl_json(accession_no, retry = 0):
    
    """
    This function was taken from the github user mentioned in the beginning of this notebook. This function \
    takes an accession number for a filing and returns the data from the filing in json format.
    """
    
    request_url = xbrl_converter_api_endpoint + "?accession-no=" + accession_no + "&token=" + '88140c3a554da7220ba7453a5e1ea2221c5f1e0e42c3293bbe75857245774e2e'

    # linear backoff in case API fails with "too many requests" error
    try:
      response_tmp = requests.get(request_url)
      xbrl_json = json.loads(response_tmp.text)
    except:
      if retry > 5:
        raise Exception('API error')
      
      # wait 500 milliseconds on error and retry
      time.sleep(0.5) 
      return get_xbrl_json(accession_no, retry + 1)

    return xbrl_json

In [86]:
list_xbrl = []
counter = 0

# loop through each accession number and append the json data into a list
for acc_no in accession_numbers:
    if counter % 10 == 0:
        print(f"processing {counter}")
    list_xbrl.append(get_xbrl_json(acc_no))
    counter += 1

processing 0
processing 10
processing 20
processing 30
processing 40
processing 50
processing 60
processing 70
processing 80
processing 90
processing 100
processing 110
processing 120
processing 130
processing 140
processing 150
processing 160
processing 170
processing 180
processing 190
processing 200
processing 210
processing 220
processing 230
processing 240
processing 250
processing 260
processing 270
processing 280
processing 290
processing 300
processing 310
processing 320
processing 330
processing 340
processing 350
processing 360
processing 370
processing 380
processing 390
processing 400
processing 410
processing 420
processing 430
processing 440
processing 450
processing 460
processing 470
processing 480


In [87]:
cleaned_list_xbrl = []

# create a cleaned list for xbrl data because some filings have no xbrl data to parse
for xbrl in list_xbrl:
    if 'message' in xbrl.keys():
        pass
    else:
        cleaned_list_xbrl.append(xbrl)

In [88]:
len(list_xbrl)

489

In [89]:
len(cleaned_list_xbrl)

291

In [91]:
# write list to binary file
def write_list(a_list):
    
    """
    This function takes in a list and dumps the contents of the list into a pickled file.
    """
    
    # store list in binary file so 'wb' mode
    with open('./data/listfile_mystocks', 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')
    fp.close()    

In [92]:
write_list(cleaned_list_xbrl)

Done writing list into a binary file


In [93]:
# Read list to memory
def read_list(path):
    
    """
    This function reads a pickled file for a list into memory.
    """
    
    # for reading also binary mode is important
    with open(path, 'rb') as fp:
        n_list = pickle.load(fp)
    fp.close()
    return n_list

In [94]:
# list of names
loaded_xbrl = read_list('./data/listfile_mystocks')
len(loaded_xbrl)

291

In [95]:
# create a stock mapper instance so I can pull ticker symbols for stocks
mapper = StockMapper()
map_stock = mapper.cik_to_tickers

In [96]:
def get_sector(ticker):
    
    """
    This function takes the ticker for a public company as a string and returns the company's sector.
    """
    
    print(ticker)
    tick = yf.Ticker(ticker)
    return tick.get_info()['sector']

In [97]:
def stock_10yr(ticker, year, count=0):
    
    """
    This function takes the ticker for a public company and a start year to begin tracking the company. 
    The function returns the overall 10 year return to shareholders as a percentage.
    """
    
    print(ticker, year)
    tick = yf.Ticker(ticker)
    try:
        start_price = tick.history(start=f'{year}-01-01')['Close'][0]
    except:
        if count > 5:
            raise Exception('yfinance not doing its job')
        time.sleep(5)
        return stock_10yr(ticker, year, count+1)
    end_year = str(int(year)+10)
    try:
        final_price = tick.history(start=f'{end_year}-01-01')['Close'][0]
    except: 
        if count > 5:
            raise Exception('yfinance not doing its job')
        time.sleep(5) 
        return stock_10yr(ticker, year, count+1)
        
    perc_return = round(((final_price - start_price) / start_price), 5)*100
    return perc_return

In [98]:
def spy_10yr(year):
    
    """
    This function takes in a year and returns the S&P500 10 year return from that year.
    """
    
    spy = yf.Ticker('SPY')
    start_price = spy.history(start=f'{year}-01-01')['Close'][0]
    end_year = str(int(year)+10)
    final_price = spy.history(start=f'{end_year}-01-01')['Close'][0]
    perc_return = round(((final_price - start_price) / start_price), 5)*100
    return perc_return

In [100]:
def financials(list_of_xbrl):
    
    """
    This function takes in a list of json data for various filings and returns a dataframe of all \
    relevant financial statistics for every company from the list. The function combinese dataframes \
    for the balance sheet, income statement, and cash flow statement. This function also creates a column \ 
    for the stock ticker symbol.
    """
    
    balance_full_list = []

    for xbrl in list_of_xbrl:
        try:
            balance_full_list.append(get_balance_sheet(xbrl))
        except:
            continue   
    balance_full_df = pd.concat(balance_full_list)
    # pull relevant columns for analysis
    balance_df_final = balance_full_df[['year', 'CIK', 'AssetsCurrent', 'LiabilitiesCurrent']]
    balance_df_final.dropna(inplace=True)
    
    income_full_list = []

    for xbrl in list_of_xbrl:
        try:
            income_full_list.append(get_income_statement(xbrl))
        except:
            continue   
    income_full_df = pd.concat(income_full_list)
    # pull relevant columns for analysis
    income_df_final = income_full_df[['OperatingIncomeLoss', 'NetIncomeLoss', 'CIK', 'year']]
    income_df_final.dropna(inplace=True)
    
    cashflow_full_list = []

    for xbrl in list_of_xbrl:
        try:
            cashflow_full_list.append(get_cash_flow_statement(xbrl))
        except:
            continue   
    cashflow_full_df = pd.concat(cashflow_full_list)
    # pull relevant columns for analysis
    cashflow_df_final = cashflow_full_df[['year', 'CIK', 'NetCashProvidedByUsedInOperatingActivities', 'NetCashProvidedByUsedInInvestingActivities', 'NetCashProvidedByUsedInFinancingActivities', 'CashAndCashEquivalentsPeriodIncreaseDecrease']]
    final_financials = balance_df_final.merge(cashflow_df_final, how='inner', on=['CIK', 'year']).merge(income_df_final, how='inner', on=['CIK', 'year'])
    final_financials['ticker'] = final_financials['CIK'].map(map_stock)
    final_financials.dropna(inplace=True)
    
    final_financials['ticker'] = final_financials['ticker'].map(str).str.slice(2, -2)
    final_financials['ticker'] = final_financials['ticker'].apply(lambda row: row if len(str(row)) <= 4 else 0)
    final_financials = final_financials[final_financials['ticker'] != 0]
    
    return final_financials
    

In [101]:
df_final = financials(loaded_xbrl)

In [103]:
df_final

,year,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss,ticker
22,2016,0001512673,1001425000,577464000,23131000,-122733000,90741000,-9299000,-170453000,-171590000,SQ
45,2016,0001595974,391577000,218437000,60120000,-37116000,10077000,32924000,-24897000,-18053000,MGNI
46,2015,0001595974,365707000,250163000,76856000,-72861000,15468000,19303000,-5598000,422000,MGNI
47,2014,0001595974,237977000,154402000,6645000,-23123000,83794000,67240000,-18778000,-18673000,MGNI
51,2013,0001141391,10950000000,6032000000,4135000000,-4000000,-2629000000,1547000000,920000000,623000000,MA
52,2012,0001141391,9357000000,4906000000,2948000000,-2839000000,-1798000000,-1682000000,899000000,605000000,MA
53,2011,0001141391,7741000000,4217000000,2684000000,-748000000,-1215000000,667000000,-10000000,19000000,MA
73,2010,0001053507,1404174000,542195000,1020977000,-1300902000,910330000,636670000,203441000,83514000,AMT
77,2014,0001403161,9562000000,6006000000,7205000000,-941000000,-6478000000,-215000000,7697000000,5438000000,V
78,2013,0001403161,7822000000,4335000000,3022000000,-1164000000,-1746000000,112000000,7239000000,4980000000,V


In [44]:
# create a list of years that data was extracted from
yrs_list = [2009, 2010, 2011, 2012]

In [46]:
# create a list of S&P 500 returns for each year
spy_returns = [spy_10yr(year) for year in yrs_list]

In [53]:
# zip the two lists together into a dictionary
spy_dict = dict(zip(yrs_list, spy_returns))

In [54]:
spy_dict

{2009: 230.065,
 2010: 250.11899999999997,
 2011: 254.2,
 2012: 353.76500000000004}

In [56]:
# create a new column that has the S&P 500 10 year returns based on the year of financial statistics
df['SPY'] = df['year'].map(spy_dict)

In [92]:
df.head()

,year,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss,ticker,SPY
0,2009,1140859,9953753000,9479610000,783763000,-1.483490e+08,-504160000,131254000,883749000,503397000,ABC,230.065
1,2009,829224,2035800000,1581000000,1389000000,-4.211000e+08,-642200000,330000000,562000000,390800000,SBUX,230.065
2,2009,796343,2473624000,844553000,1117772000,-1.497096e+09,477658000,113037000,690513000,386508000,ADBE,230.065
3,2009,875045,2480595000,714942000,1074898000,-3.950310e+08,-724151000,-44284000,1295427000,970132000,BIIB,230.065
4,2009,100885,3680000000,2682000000,3234000000,-2.175000e+09,-458000000,601000000,3392000000,1898000000,UNP,230.065


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 12 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Unnamed: 0                                    519 non-null    int64  
 1   year                                          519 non-null    int64  
 2   CIK                                           519 non-null    int64  
 3   AssetsCurrent                                 519 non-null    int64  
 4   LiabilitiesCurrent                            519 non-null    int64  
 5   NetCashProvidedByUsedInOperatingActivities    519 non-null    int64  
 6   NetCashProvidedByUsedInInvestingActivities    519 non-null    float64
 7   NetCashProvidedByUsedInFinancingActivities    519 non-null    int64  
 8   CashAndCashEquivalentsPeriodIncreaseDecrease  519 non-null    int64  
 9   OperatingIncomeLoss                           519 non-null    int

In [43]:
df.groupby('year').count()

,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss,ticker
year,,,,,,,,,,
2009,10,10,10,10,10,10,10,10,10,10
2010,17,17,17,17,17,17,17,17,17,17
2011,119,119,119,119,119,119,119,119,119,119
2012,64,64,64,64,64,64,64,64,64,64


In [33]:
# drop duplicate rows as there were some cases with over 10 duplicate rows
df.drop_duplicates(inplace=True)

In [75]:
df.head(20)

,year,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss,ticker,SPY
0,2009,1140859,9953753000,9479610000,783763000,-1.483490e+08,-504160000,131254000,883749000,503397000,ABC,230.065
1,2009,829224,2035800000,1581000000,1389000000,-4.211000e+08,-642200000,330000000,562000000,390800000,SBUX,230.065
2,2009,796343,2473624000,844553000,1117772000,-1.497096e+09,477658000,113037000,690513000,386508000,ADBE,230.065
3,2009,875045,2480595000,714942000,1074898000,-3.950310e+08,-724151000,-44284000,1295427000,970132000,BIIB,230.065
4,2009,100885,3680000000,2682000000,3234000000,-2.175000e+09,-458000000,601000000,3392000000,1898000000,UNP,230.065
5,2009,1022079,1679425000,1059212000,997418000,-1.959040e+08,-521204000,280310000,1359111000,729111000,DGX,230.065
6,2009,823768,3010000000,2901000000,2362000000,-1.250000e+09,-457000000,660000000,1887000000,994000000,WM,230.065
7,2009,1306830,2856000000,1607000000,596000000,3.100000e+07,-112000000,578000000,290000000,488000000,CE,230.065
8,2009,7332,564501000,536416000,1359376000,-1.780604e+09,238135000,-183093000,-34966000,-35650000,SWN,230.065
9,2009,1018963,1997600000,624600000,218500000,-4.537000e+08,474100000,238900000,92700000,31700000,ATI,230.065


Given the yfinance package is not the most reliable, I needed to subset my data before making repeated API calls. By doing this, if the code failed to pull relevant data from Yahoo Finance, I only lost a small portion of time. Each df listed below is a subset of the final dataframe that I created

In [223]:
df1 = df.iloc[:5]
df2 = df.iloc[5:25]

In [145]:
df3 = df.iloc[25:40]
df5 = df.iloc[40:55]
df6 = df.iloc[55:70]
df7 = df.iloc[70:85]

In [211]:
df8 = df.iloc[100:150]
df9 = df.iloc[150:]

In [119]:
df4 = df.iloc[85:100]

Next, I created multiple lists of tuples that contain both a stock's ticker symbol as well as the year. By doing this, I can loop through the list to create a list of all the 10 year returns for these stocks. I initially tried to do this in a function, but the yfinance package took significantly too long.

In [98]:
tick_year2 = list(zip(df2['ticker'], df2['year']))

In [146]:
tick_year3 = list(zip(df3['ticker'], df3['year']))

In [122]:
tick_year4 = list(zip(df4['ticker'], df4['year']))

In [155]:
tick_year5 = list(zip(df5['ticker'], df5['year']))
tick_year6 = list(zip(df6['ticker'], df6['year']))
tick_year7 = list(zip(df7['ticker'], df7['year']))

In [202]:
tick_year8 = list(zip(df8['ticker'], df8['year']))

In [213]:
tick_year9 = list(zip(df9['ticker'], df9['year']))

In [100]:
returnslist2 = [stock_10yr(ticker, year) for (ticker, year) in tick_year]

In [148]:
returnslist3 = []

for ticker, year in tick_year3:
    returnslist3.append(stock_10yr(ticker, year))

CHD 2010
OMC 2010
HUBG 2010
IART 2010
WW 2011
CRM 2011
POLY 2011
MOS 2011
MLKN 2011
TECH 2011
SCHN 2011
PSMT 2011
ASYS 2011
BEN 2011
SPH 2011


In [133]:
returnslist4 = []

for ticker, year in tick_year4:
    returnslist4.append(stock_10yr(ticker, year))

TBI 2011
TBI 2011
ABG 2011
ABG 2011
MGTI 2011
MGTI 2011
SIGA 2011
SIGA 2011
AUTO 2011
AUTO 2011
PRFT 2011
PRFT 2011
PDCE 2011
PDCE 2011
RNWK 2011
RNWK 2011
FSLR 2011
FSLR 2011
VHC 2011
VHC 2011
VHC 2011
VHC 2011
MDRX 2011
MDRX 2011
NC 2011
NC 2011
HCCI 2011
HCCI 2011
CXDO 2011
CXDO 2011


In [161]:
returnslist5 = []

for ticker, year in tick_year5:
    returnslist5.append(stock_10yr(ticker, year))

MATW 2011
SANM 2011
SIF 2011
KNWN 2011
BECN 2011
FRPH 2011
HPQ 2011
CBAT 2011
CIEN 2011
MFCO 2011
RGCO 2011
IESC 2011
PGLO 2011
JOUT 2011
EMKR 2011


In [169]:
returnslist6 = []

for ticker, year in tick_year6:
    returnslist6.append(stock_10yr(ticker, year))

DYSL 2011
GSTC 2011
- GSTC: No data found for this date range, symbol may be delisted
GSTC 2011
- GSTC: No data found for this date range, symbol may be delisted
GSTC 2011
- GSTC: No data found for this date range, symbol may be delisted
GSTC 2011
GSTC 2011
- GSTC: No data found for this date range, symbol may be delisted
GSTC 2011
JOB 2011
BDL 2011
OPXS 2011
GSTX 2011
BUUZ 2011
PLAB 2011
STLY 2011
PBSV 2011
INLX 2011
GFGY 2011
SNX 2011
LEVI 2011
DHX 2011


In [173]:
returnslist7 = []

for ticker, year in tick_year7:
    returnslist7.append(stock_10yr(ticker, year))

VG 2011
MAR 2011
HAL 2011
MA 2011
MPAD 2011
CCI 2011
CE 2011
AMKR 2011
INTC 2011
EAF 2011
FYBR 2011
CTXS 2011
KO 2011
GPK 2011
ANSS 2011


In [203]:
returnslist8 = []

for ticker, year in tick_year8:
    returnslist8.append(stock_10yr(ticker, year))

CEMI 2011
VSEC 2011
PXLW 2011
LXRX 2011
BYD 2011
ACU 2011
GERN 2011
TREX 2011
PRPH 2011
HHS 2011
NLS 2011
EGLE 2011
STRL 2011
CUTR 2011
BBSI 2011
MODV 2011
STXS 2011
CLSN 2011
PESI 2011
EBIX 2011
DXYN 2011
IMUC 2011
AMNL 2011
LOAN 2011
ADYX 2011
CLTH 2011
UCIX 2011
UCIX 2011
PAR 2011
USIO 2011
URBN 2012
GAXY 2011
SCVL 2012
NNAX 2011
CONN 2012
LTUM 2011
HQI 2011
LAKE 2012
BUDZ 2011
AGSS 2011
CAMP 2012
MICR 2011
NVEC 2012
PPIH 2012
LBSR 2012
ZWS 2012
PBH 2012
SODI 2012
DBRM 2012
ESTE 2012


In [215]:
returnslist9 = []

for ticker, year in tick_year9:
    returnslist9.append(stock_10yr(ticker, year))

TESS 2012
MOD 2012
RNVA 2012
QMCO 2012
NXTP 2012
DAKT 2012
CIDM 2012
UUU 2012
PFIE 2012
MEI 2012
FRHC 2012
ELTP 2012
AMWD 2012
MICS 2012
TTEC 2011
MOS 2012
TENX 2012
RGP 2012
GBLX 2012
GPN 2012
OTEX 2012
STX 2012
PG 2012
NAUH 2012
AIT 2012
VSYM 2011
LTRX 2012
HAIN 2012
TAYD 2012
TECH 2012
AEHR 2012
SCSC 2012
AOSL 2012
CTIB 2011
PSTV 2011
SANW 2012
DNAX 2011
EVI 2012
MPAA 2012
WESC 2012
ATRX 2011
SGLY 2012
SCHN 2012
SRRE 2011
IVZ 2011
CUII 2012
NNVC 2012
HMLA 2012
SANM 2012
SBUX 2012
TDG 2012
BECN 2012
SPH 2012
MATW 2012
EACO 2012
GRYN 2012
MFCO 2012
CIEN 2012


With all of the 10 year returns now generated, I can insert a new column into each subset of data containing the 10 year return percentage. 

In [227]:
df1.insert(loc=11, column='10yrReturns', value=returns)

In [234]:
df2.insert(loc=11, column='10yrReturns', value=returnslist2)

In [149]:
df3.insert(loc=11, column='10yrReturns', value=returnslist3)

In [136]:
df4.insert(loc=11, column='10yrReturns', value=returnslist4)

In [162]:
df5.insert(loc=11, column='10yrReturns', value=returnslist5)

In [170]:
df6.insert(loc=11, column='10yrReturns', value=returnslist6)

In [177]:
df7.insert(loc=11, column='10yrReturns', value=returnslist7)

In [233]:
df8.insert(loc=11, column='10yrReturns', value=returnslist8)

In [216]:
df9.insert(loc=11, column='10yrReturns', value=returnslist9)

In [20]:
# saving off all dataframes as CSV files to ensure I won't need to run this code again
df1.to_csv('./data/df1')

In [235]:
df2.to_csv('./data/df2')

In [150]:
df3.to_csv('./data/df3')

In [140]:
df4.to_csv('./data/df4')

In [164]:
df5.to_csv('./data/df5')

In [172]:
df6.to_csv('./data/df6')

In [179]:
df7.to_csv('./data/df7')

In [236]:
df8.to_csv('./data/df8')

In [218]:
df9.to_csv('./data/df9')

In [200]:
df.to_csv('./data/baseline_df')

In [237]:
# combine all of the dataframes together 
full_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], axis=0, ignore_index=True)

In [241]:
# create our target column that is simply a Boolean for whether or not the stock's 10 year return outperformed the 
# S&P 500 10 year return
full_df['marketBeat?'] = (full_df['10yrReturns'] > full_df['SPY']).astype(int)

In [242]:
full_df

,year,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss,ticker,10yrReturns,SPY,marketBeat?
0,2009,1140859,9953753000,9479610000,783763000,-1.483490e+08,-504160000,131254000,883749000,503397000,ABC,382.872,230.065,1
1,2009,829224,2035800000,1581000000,1389000000,-4.211000e+08,-642200000,330000000,562000000,390800000,SBUX,1404.256,230.065,1
2,2009,796343,2473624000,844553000,1117772000,-1.497096e+09,477658000,113037000,690513000,386508000,ADBE,875.543,230.065,1
3,2009,875045,2480595000,714942000,1074898000,-3.950310e+08,-724151000,-44284000,1295427000,970132000,BIIB,528.356,230.065,1
4,2009,100885,3680000000,2682000000,3234000000,-2.175000e+09,-458000000,601000000,3392000000,1898000000,UNP,578.284,230.065,1
5,2009,1022079,1679425000,1059212000,997418000,-1.959040e+08,-521204000,280310000,1359111000,729111000,DGX,81.640,230.065,0
6,2009,823768,3010000000,2901000000,2362000000,-1.250000e+09,-457000000,660000000,1887000000,994000000,WM,265.080,230.065,1
7,2009,1306830,2856000000,1607000000,596000000,3.100000e+07,-112000000,578000000,290000000,488000000,CE,692.905,230.065,1
8,2009,7332,564501000,536416000,1359376000,-1.780604e+09,238135000,-183093000,-34966000,-35650000,SWN,-88.035,230.065,0
9,2009,1018963,1997600000,624600000,218500000,-4.537000e+08,474100000,238900000,92700000,31700000,ATI,-2.913,230.065,0


In [244]:
# save off final dataframe as a CSV
full_df.to_csv('./data/modeling_df')

In [2]:
final_df = pd.read_csv('./data/modeling_df')

In [4]:
final_df.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [5]:
final_df.head()

,year,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss,ticker,10yrReturns,SPY,marketBeat?
0,2009,1140859,9953753000,9479610000,783763000,-1.483490e+08,-504160000,131254000,883749000,503397000,ABC,382.872,230.065,1
1,2009,829224,2035800000,1581000000,1389000000,-4.211000e+08,-642200000,330000000,562000000,390800000,SBUX,1404.256,230.065,1
2,2009,796343,2473624000,844553000,1117772000,-1.497096e+09,477658000,113037000,690513000,386508000,ADBE,875.543,230.065,1
3,2009,875045,2480595000,714942000,1074898000,-3.950310e+08,-724151000,-44284000,1295427000,970132000,BIIB,528.356,230.065,1
4,2009,100885,3680000000,2682000000,3234000000,-2.175000e+09,-458000000,601000000,3392000000,1898000000,UNP,578.284,230.065,1


In [8]:
df1 = pd.read_csv('./data/df1')

In [9]:
df2 = pd.read_csv('./data/df2')

In [10]:
df3 = pd.read_csv('./data/df3')

In [11]:
df4 = pd.read_csv('./data/df4')

In [12]:
df5 = pd.read_csv('./data/df5')

In [13]:
df6 = pd.read_csv('./data/df6')

In [14]:
df7 = pd.read_csv('./data/df7')

In [15]:
df8 = pd.read_csv('./data/df8')

In [16]:
df9 = pd.read_csv('./data/df9')

In [18]:
df1['sector'] = df1['ticker'].map(get_sector)

ABC
SBUX
ADBE
BIIB
UNP


In [47]:
# append stock sectors into a dictionary if the ticker symbol is not already in the dictionary
for ticker in final_df['ticker']:
    if ticker in sector_dict.keys() or ticker=='PGLO':
        pass
    else:
        sector_dict[ticker] = get_sector(ticker)

JOUT
EMKR
DYSL
GSTC
JOB
BDL
OPXS


In [49]:
len(sector_dict)

191

In [19]:
df1

,Unnamed: 0,year,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss,ticker,10yrReturns,SPY,sector
0,0,2009,1140859,9953753000,9479610000,783763000,-1.483490e+08,-504160000,131254000,883749000,503397000,ABC,382.872,230.065,Healthcare
1,1,2009,829224,2035800000,1581000000,1389000000,-4.211000e+08,-642200000,330000000,562000000,390800000,SBUX,1404.256,230.065,Consumer Cyclical
2,2,2009,796343,2473624000,844553000,1117772000,-1.497096e+09,477658000,113037000,690513000,386508000,ADBE,875.543,230.065,Technology
3,3,2009,875045,2480595000,714942000,1074898000,-3.950310e+08,-724151000,-44284000,1295427000,970132000,BIIB,528.356,230.065,Healthcare
4,4,2009,100885,3680000000,2682000000,3234000000,-2.175000e+09,-458000000,601000000,3392000000,1898000000,UNP,578.284,230.065,Industrials


In [50]:
# create a sector column in our final dataframe by mapping the sector dictionary to the ticker column
final_df['sector'] = final_df['ticker'].map(sector_dict)

In [55]:
final_df.head()

,year,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss,ticker,10yrReturns,SPY,marketBeat?,sector
0,2009,1140859,9953753000,9479610000,783763000,-1.483490e+08,-504160000,131254000,883749000,503397000,ABC,382.872,230.065,1,Healthcare
1,2009,829224,2035800000,1581000000,1389000000,-4.211000e+08,-642200000,330000000,562000000,390800000,SBUX,1404.256,230.065,1,Consumer Cyclical
2,2009,796343,2473624000,844553000,1117772000,-1.497096e+09,477658000,113037000,690513000,386508000,ADBE,875.543,230.065,1,Technology
3,2009,875045,2480595000,714942000,1074898000,-3.950310e+08,-724151000,-44284000,1295427000,970132000,BIIB,528.356,230.065,1,Healthcare
4,2009,100885,3680000000,2682000000,3234000000,-2.175000e+09,-458000000,601000000,3392000000,1898000000,UNP,578.284,230.065,1,Industrials


In [56]:
# save our final dataframe to be used in modeling
final_df.to_csv('./data/modeling_df2')